In [1]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

In [2]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

def V(num_qubits: int):
    '''
        Create a ansatz V with n qubits and assign its parameters
    '''
    #Assign random parameter
    circuit = ansatz.stargraph(num_qubits=num_qubits)
    num_params = circuit.num_parameters
    x0 = 2 * np.pi * np.random.random(num_params)
    circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))
    return circuit

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calculate_rho_3 (rho, unitary_matrix, kraus_operators):
    '''
        rho -E(rho) = U @ rho @ U(dagger)-> rho' -> U(dagger) @ U @ rho @ U(dagger) @ U -> rho
        sum(K @ U @ rho @ U(dagger) @ K(dagger)) //rho'' = rho
    '''
    rho_2 = unitary_matrix  @ rho @ np.transpose(np.conjugate(unitary_matrix))
    rho_3 = sum(K @ rho_2 @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    return rho_3

def create_kraus_operators(unitary_matrix):
    '''
        Create a set of Kraus Operators from the input unitary matrix, using QR decomposition
    '''
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #Q: a 2^N x 2^N matrix, N is the number of qubits
    for q in Q:
        q = np.expand_dims(q, 1)
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))
    return tf.convert_to_tensor(kraus_ops)

def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """
    rho_2 = tf.linalg.sqrtm((tf.linalg.sqrtm(rho)) @ (rho))

    # Cast to a supported type
    real_part = tf.math.real(rho_2)
    imaginary_part = tf.math.imag(rho_2)

    # Check for NaNs in both real and imaginary parts
    contains_nan_real = tf.reduce_any(tf.math.is_nan(real_part))
    contains_nan_imag = tf.reduce_any(tf.math.is_nan(imaginary_part))

    contains_nan = contains_nan_real or contains_nan_imag

    if contains_nan == True:
        rho_2 = rho

    return tf.linalg.trace(
            rho_2
            @ (tf.linalg.sqrtm(sigma))
        )

def frobenius_norm(rho, sigma):
    """
    Compute the Frobenius norm between two matrices.

    Parameters:
    rho (numpy.ndarray): The first matrix.
    sigma (numpy.ndarray): The second matrix.

    Returns:
    float: The Frobenius norm between rho and rho_prime.
    """

    # Ensure the matrices are TensorFlow tensors
    rho = tf.convert_to_tensor(rho, dtype=tf.complex128)
    sigma = tf.convert_to_tensor(sigma, dtype=tf.complex128)

    # Compute the difference between the matrices
    diff = rho - sigma

    # Compute the Frobenius norm
    #norm = tf.linalg.normalize(diff, ord='fro')
    norm = tf.sqrt(tf.reduce_sum(tf.square(diff)))
    return norm

#cost func to compare 2 given rhos
def cost(rho, rho_3):
    return tf.square(frobenius_norm(rho, rho_3))


In [3]:
n = 3

#Create a ansatz V with N qubits
circuit = V(n)

print(circuit)

     ┌────────────┐   ┌────────────┐   
q_0: ┤ Ry(2.1819) ├─■─┤ Ry(3.4218) ├─■─
     ├────────────┤ │ └────────────┘ │ 
q_1: ┤ Ry(4.7851) ├─■────────────────┼─
     ├────────────┤                  │ 
q_2: ┤ Ry(4.6374) ├──────────────────■─
     └────────────┘                    


In [4]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data

print (unitary_matrix) #2^N x 2^N

[[-0.46959889+0.j -0.1659604 +0.j -0.43662844+0.j -0.15430835+0.j
  -0.5062228 +0.j -0.17890361+0.j -0.470681  +0.j -0.16634282+0.j]
 [ 0.1659604 +0.j -0.46959889+0.j  0.15430835+0.j -0.43662844+0.j
   0.17890361+0.j -0.5062228 +0.j  0.16634282+0.j -0.470681  +0.j]
 [-0.37691838+0.j -0.26905018+0.j  0.40538003+0.j  0.28936655+0.j
  -0.40631416+0.j -0.29003334+0.j  0.43699553+0.j  0.31193418+0.j]
 [-0.26905018+0.j  0.37691838+0.j  0.28936655+0.j -0.40538003+0.j
  -0.29003334+0.j  0.40631416+0.j  0.31193418+0.j -0.43699553+0.j]
 [ 0.5062228 +0.j  0.17890361+0.j  0.470681  +0.j  0.16634282+0.j
  -0.46959889+0.j -0.1659604 +0.j -0.43662844+0.j -0.15430835+0.j]
 [ 0.17890361+0.j -0.5062228 +0.j  0.16634282+0.j -0.470681  +0.j
  -0.1659604 +0.j  0.46959889+0.j -0.15430835+0.j  0.43662844+0.j]
 [ 0.40631416+0.j  0.29003334+0.j -0.43699553+0.j -0.31193418+0.j
  -0.37691838+0.j -0.26905018+0.j  0.40538003+0.j  0.28936655+0.j]
 [-0.29003334+0.j  0.40631416+0.j  0.31193418+0.j -0.43699553+0.j
   

In [5]:
# Initialize a set of Kraus Operators from the given circuit
kraus_operators = create_kraus_operators(unitary_matrix=unitary_matrix)

#print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))
print(unitary_matrix @ np.transpose(np.conjugate(unitary_matrix)))


[[ 1.00000000e+00+0.j  4.07180401e-18+0.j  3.23323957e-17+0.j
  -4.26947040e-17+0.j -3.69634617e-17+0.j -8.65534487e-17+0.j
   2.06930124e-17+0.j  1.83776684e-17+0.j]
 [ 4.07180401e-18+0.j  1.00000000e+00+0.j -2.32582146e-17+0.j
  -2.12821065e-17+0.j -8.04095687e-17+0.j  4.01809378e-17+0.j
  -2.88083898e-17+0.j  2.99009004e-17+0.j]
 [ 3.23323957e-17+0.j -2.32582146e-17+0.j  1.00000000e+00+0.j
  -5.94563417e-18+0.j -4.59946040e-17+0.j -2.71507204e-17+0.j
   6.72592452e-18+0.j -2.64786903e-17+0.j]
 [-4.26947040e-17+0.j -2.12821065e-17+0.j -5.94563417e-18+0.j
   1.00000000e+00+0.j -1.05194314e-17+0.j -1.84083755e-17+0.j
  -4.30699040e-17+0.j -5.43201927e-17+0.j]
 [-3.69634617e-17+0.j -8.04095687e-17+0.j -4.59946040e-17+0.j
  -1.05194314e-17+0.j  1.00000000e+00+0.j  5.09092819e-18+0.j
  -4.71672949e-19+0.j  4.27653553e-17+0.j]
 [-8.65534487e-17+0.j  4.01809378e-17+0.j -2.71507204e-17+0.j
  -1.84083755e-17+0.j  5.09092819e-18+0.j  1.00000000e+00+0.j
   3.26921426e-17+0.j  4.58941462e-18+0.j

In [6]:
#Initialize rho
rho = DensityMatrix.from_label('0' * n)
rho = tf.convert_to_tensor(rho)

#calculate rho_3
rho_3 = calculate_rho_3(rho=rho, kraus_operators=kraus_operators, unitary_matrix=unitary_matrix) 

#print(rho)
#print(rho3)


In [7]:
print(cost(rho=rho, rho_3=rho_3))

tf.Tensor((1.0994815861721623+0j), shape=(), dtype=complex128)


In [8]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

In [9]:
#Auto Diff
'''
    rho: The initial rho
    unitary_matrix: randomly generated circle in initialization step
    kraus_operators: set of kraus operators
    n: number of qubits
    alpha: learning rate (?)'''
def calculate_derivative(rho, unitary_matrix, kraus_operators, n, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators)
    with tf.GradientTape() as tape:
        y = calculate_rho_3(rho, unitary_matrix, tensorKraus)
        f = cost(rho, y) 
    
    # Get the gradient of y with respect to x
    c = tape.gradient(f, tensorKraus)

    # Reshape c to kN * N matrix
    c = tf.reshape(c, (2**n * 2**n,2**n))

    # Reshape kraus_operators to kN * N matrix
    kraus_operators = tf.reshape(kraus_operators, (2**n * 2**n,2**n))
    
    # Compute the projection term
    proj = c - kraus_operators @ (np.transpose(np.conjugate(c)) @ kraus_operators  + np.transpose(np.conjugate(kraus_operators)) @ c) / 2

    # Update the Kraus operators
    updated_kraus_operators = kraus_operators - alpha * proj
    
    # Return the updated Kraus operators
    return updated_kraus_operators



In [10]:
kraus_operators_copy = tf.identity(kraus_operators)

In [11]:
# try looping manually
for i in range (0, 1000):
    _cost = cost(rho, calculate_rho_3(rho, unitary_matrix, kraus_operators_copy))

    #Update Kraus Operators
    kraus_operators_copy=calculate_derivative(rho, unitary_matrix, kraus_operators_copy, n)

    #Reshape
    kraus_operators_copy = tf.reshape(kraus_operators_copy, (2**n,2**n,2**n))
    
    #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

    print (_cost)

tf.Tensor((1.0994815861721623+0j), shape=(), dtype=complex128)
tf.Tensor((1.0119524331103875+0j), shape=(), dtype=complex128)
tf.Tensor((0.9078986778469077+0j), shape=(), dtype=complex128)
tf.Tensor((0.7901033452423137+0j), shape=(), dtype=complex128)
tf.Tensor((0.6647949205062709+0j), shape=(), dtype=complex128)
tf.Tensor((0.5406572338008884+0j), shape=(), dtype=complex128)
tf.Tensor((0.4264429558439135+0j), shape=(), dtype=complex128)
tf.Tensor((0.3284170040569295+0j), shape=(), dtype=complex128)
tf.Tensor((0.24911618563764562+0j), shape=(), dtype=complex128)
tf.Tensor((0.18781154207641415+0j), shape=(), dtype=complex128)
tf.Tensor((0.1418753983991013+0j), shape=(), dtype=complex128)
tf.Tensor((0.10808697517622433+0j), shape=(), dtype=complex128)
tf.Tensor((0.08343672727450809+0j), shape=(), dtype=complex128)
tf.Tensor((0.06545721879159548+0j), shape=(), dtype=complex128)
tf.Tensor((0.05226780297597941+0j), shape=(), dtype=complex128)
tf.Tensor((0.0424943008496498+0j), shape=(), dtyp

In [12]:
print(sum(K @ np.transpose(np.conjugate(K)) for K in kraus_operators_copy))
print(kraus_operators_copy)

tf.Tensor(
[[ 1.86909446e+00+0.j -3.15825021e-02+0.j -1.17061662e-01+0.j
   3.71282381e-02+0.j  7.38884379e-02+0.j -2.01890225e-02+0.j
   8.40399354e-02+0.j -2.26999877e-02+0.j]
 [-3.15825021e-02+0.j  8.70819648e-01+0.j -1.65110657e-02+0.j
   7.91642824e-02+0.j  4.37406550e-03+0.j -6.20787076e-02+0.j
   1.43690868e-03+0.j -7.93502480e-02+0.j]
 [-1.17061662e-01+0.j -1.65110657e-02+0.j  8.71547736e-01+0.j
   2.95021555e-03+0.j  7.78549937e-02+0.j -1.92126244e-03+0.j
   7.94108318e-02+0.j -4.72978530e-03+0.j]
 [ 3.71282381e-02+0.j  7.91642824e-02+0.j  2.95021555e-03+0.j
   8.60923370e-01+0.j -1.99664335e-03+0.j  7.85943154e-02+0.j
  -4.98113512e-03+0.j  7.31783957e-02+0.j]
 [ 7.38884379e-02+0.j  4.37406550e-03+0.j  7.78549937e-02+0.j
  -1.99664335e-03+0.j  8.95826446e-01+0.j  3.19772850e-04+0.j
  -9.41840828e-02+0.j  1.73477746e-02+0.j]
 [-2.01890225e-02+0.j -6.20787076e-02+0.j -1.92126244e-03+0.j
   7.85943154e-02+0.j  3.19772850e-04+0.j  8.81354970e-01+0.j
   1.74593563e-02+0.j -6.73852

In [13]:
#Check the output of sum(K @ U @ rho @ U(dagger) @ K(dagger))  - suppose to be rho
rho_out_1 = calculate_rho_3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_copy)

In [14]:
#Compare with rho
print(compilation_trace_fidelity(rho,rho_out_1))

tf.Tensor((0.9989167338873859+0j), shape=(), dtype=complex128)


In [15]:
print (rho_out_1)

tf.Tensor(
[[ 9.97834642e-01+0.j -1.31236789e-05+0.j -1.11981909e-05+0.j
   1.83755014e-06+0.j  1.18104742e-05+0.j -3.11568815e-06+0.j
   1.18725175e-05+0.j -5.62235890e-06+0.j]
 [-1.31236789e-05+0.j  6.84001993e-04+0.j  1.07258791e-04+0.j
   1.94922799e-04+0.j  1.08092500e-05+0.j -6.25726395e-05+0.j
   4.04727318e-05+0.j  3.46509628e-05+0.j]
 [-1.11981909e-05+0.j  1.07258791e-04+0.j  1.82961211e-04+0.j
   2.84188468e-04+0.j -1.99270000e-06+0.j -4.78850491e-05+0.j
   2.60881376e-05+0.j -1.99930174e-05+0.j]
 [ 1.83755014e-06+0.j  1.94922799e-04+0.j  2.84188468e-04+0.j
   6.79737212e-04+0.j -5.42737764e-05+0.j -4.75801933e-05+0.j
  -2.87206424e-05+0.j  4.91781969e-05+0.j]
 [ 1.18104742e-05+0.j  1.08092500e-05+0.j -1.99270000e-06+0.j
  -5.42737764e-05+0.j  2.65523270e-04+0.j  2.59729835e-04+0.j
   1.50153649e-04+0.j -6.46928733e-05+0.j]
 [-3.11568815e-06+0.j -6.25726395e-05+0.j -4.78850491e-05+0.j
  -4.75801933e-05+0.j  2.59729835e-04+0.j  7.26050902e-04+0.j
  -6.78056932e-05+0.j -2.13907